<h1>프로젝트 개요</h1>

컴퓨터 비전 프로젝트를 지나, 자연어 처리 프로젝트에 오신 스프린터 여러분들을 환영합니다! 

- 지금까지 학습하신 자연어처리 및 LLM 지식들을 토대로, **RAG 시스템을 구축하여 복잡한 형태의 기업 및 정부 제안요청서(RFP) 내용을 효과적으로 추출하고 요약하여 필요한 정보를 제공**하는 서비스를 만들어봅시다.
- 여러분들을 **B2G 입찰지원 전문 컨설팅 스타트업 – ‘입찰메이트’**의 엔지니어링 팀이라고 가정해 볼게요.
    - ‘입찰메이트’는 공공입찰 컨설팅 서비스를 제공하는 스타트업입니다.
    - 하루 수백건의 RFP(제안요청서)가 나라장터 등에서 올라오게 되는데, 한 요청서당 수십 페이지가 넘는 걸 기업 담당자들이 일일이 다 읽어볼 순 없겠죠. ‘입찰메이트’는 쏟아져나오는 요청서 가운데서 고객사에게 딱 알맞는 입찰 기회를 빠르게 찾아 고객사에게 추천하는 비즈니스를 하고 있습니다. 따라서 ‘입찰메이트’의 컨설턴트들은 **RFP의 주요 요구 조건, 대상 기관, 예산, 제출 방식 등** 중요한 정보를 핵심만 빠르게 파악한 뒤, 고객사들에게 추천하여 컨설팅까지 이어질 수 있는 기회를 만들어야합니다.
    - ‘입찰메이트’의 엔지니어링 팀은 **사용자의 요청에 따라 RFP 문서의 내용을 효과적으로 추출하고 요약하여 필요한 정보를 제공할 수 있는 사내 RAG 시스템을 구현**하는 미션을 부여 받았습니다. 그렇게 되면 ‘입찰메이트’의 컨설턴트들이 수십 페이지가 넘어가는 제안서를 일일이 들여다볼 일은 없어지고, 컨설팅 업무에 최대한 집중할 수 있겠네요.
- 여러분들은 100개의 실제 RFP 문서와 각각의 메타데이터를 제공받을 예정입니다. 다양한 자연어 처리 모델들로 실험하여 해당 문서들의 내용을 바탕으로 Q&A를 할 수 있는 시스템을 구축해보세요. 그리고 평가 방식이나 지표를 팀 별로 직접 선정하여 성능을 평가해보세요. 여러가지 의사 결정 과정이 모두 보고서와 발표에 드러나야 합니다.

<h3>1. 데이터 전처리</h3>

1.1 “배치 추출”로 전체 HWP를 processed에 저장

WSL 기반으로 HWP 원문 텍스트 추출 파이프라인을 구축했고,

총 96개 HWP 중 95개를 성공적으로 텍스트화했다.

1개 파일은 XML 파싱 오류로 추출 실패하여, CSV의 텍스트 컬럼을 fallback으로 사용하도록 설계했다.
→ “실패 감지 + 대체 경로”까지 포함한 안정적 인입 구조 완성.

1.2 PDF → TXT 배치 추출

In [1]:
from pathlib import Path
import pdfplumber
import fitz  # PyMuPDF
import re

ROOT = Path(r"C:\Users\jhye3\heum\dev\team1_middle_project")
PDF_DIR = ROOT / "data" / "raw" / "files"
OUT_DIR = ROOT / "data" / "processed" / "pdf_txt"
LOG_DIR = ROOT / "data" / "processed" / "pdf_logs"

OUT_DIR.mkdir(parents=True, exist_ok=True)
LOG_DIR.mkdir(parents=True, exist_ok=True)

def clean_text(t: str) -> str:
    if not t:
        return ""
    t = re.sub(r"\n{3,}", "\n\n", t)
    return t.strip()

def extract_pdf_pdfplumber(pdf_path: Path) -> str:
    texts = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            txt = page.extract_text() or ""
            texts.append(txt)
    return clean_text("\n".join(texts))

def extract_pdf_pymupdf(pdf_path: Path) -> str:
    doc = fitz.open(pdf_path)
    texts = [page.get_text() for page in doc]
    return clean_text("\n".join(texts))

failed = []

for pdf in PDF_DIR.glob("*.pdf"):
    out = OUT_DIR / f"{pdf.stem}.txt"
    try:
        text = extract_pdf_pdfplumber(pdf)
        if len(text) < 500:  # 너무 짧으면 fallback
            text = extract_pdf_pymupdf(pdf)
        if len(text) < 200:
            raise ValueError("extracted text too short")
        out.write_text(text, encoding="utf-8", errors="ignore")
        print("OK:", pdf.name, len(text))
    except Exception as e:
        failed.append((pdf.name, str(e)))
        (LOG_DIR / f"{pdf.stem}.err").write_text(str(e), encoding="utf-8")
        print("FAILED:", pdf.name, e)

print("\nsummary")
print("pdf ok:", len(list(OUT_DIR.glob('*.txt'))))
print("pdf failed:", len(failed))


OK: 고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf 188943
OK: 기초과학연구원_2025년도 중이온가속기용 극저온시스템 운전 용역.pdf 45078
OK: 서울시립대학교_[사전공개] 학업성취도 다차원 종단분석 통합시스템 1차.pdf 109430
OK: 서울특별시_2024년 지도정보 플랫폼 및 전문활용 연계 시스템 고도화 용.pdf 114796

summary
pdf ok: 4
pdf failed: 0


지금 상태 최종 요약 (체크포인트)

✅ HWP 원문 TXT: 95개 성공 / 1개 fallback

✅ PDF 원문 TXT: 4개 전부 성공

✅ CSV 텍스트: fallback용 확보
→ 포맷별 인입 전략 + 대체 경로까지 갖춘, 실무형 파이프라인 완성

1.3 통합 로더 (HWP → PDF → CSV fallback)

HWP 있으면 HWP 쓰고 → 없으면 PDF → 그것도 없으면 CSV
왜냐면 데이터가 가장 많고(정보량이 크고) 정확한 것부터 쓰는 게 맞기 때문

In [2]:
import pandas as pd
from pathlib import Path
import re

ROOT = Path(r"C:\Users\jhye3\heum\dev\team1_middle_project")
CSV_PATH = ROOT / "data/raw/data_list.csv"
HWP_TXT_DIR = ROOT / "data/processed/hwp_txt"
PDF_TXT_DIR = ROOT / "data/processed/pdf_txt"

df = pd.read_csv(CSV_PATH, encoding="utf-8-sig")

def clean_text(t: str) -> str:
    if not isinstance(t, str):
        return ""
    # 레이아웃 태그 제거
    t = re.sub(r"(?m)^\s*<[^>]+>\s*$", "", t)
    # 과도한 빈줄 정리
    t = re.sub(r"\n{3,}", "\n\n", t)
    return t.strip()

def load_best_text(row):
    stem = Path(str(row["파일명"])).stem

    # 1) HWP 원문
    hwp_txt = HWP_TXT_DIR / f"{stem}.txt"
    if hwp_txt.exists():
        t = clean_text(hwp_txt.read_text(encoding="utf-8", errors="ignore"))
        if len(t) >= 1000:
            return t, "hwp_txt"

    # 2) PDF 원문
    pdf_txt = PDF_TXT_DIR / f"{stem}.txt"
    if pdf_txt.exists():
        t = clean_text(pdf_txt.read_text(encoding="utf-8", errors="ignore"))
        if len(t) >= 800:
            return t, "pdf_txt"

    # 3) CSV fallback
    return clean_text(row.get("텍스트", "")), "csv_text"

docs = []
for _, r in df.iterrows():
    text, source = load_best_text(r)
    docs.append({
        "doc_id": f"{r['공고 번호']}_{int(r['공고 차수']) if pd.notna(r['공고 차수']) else 0}",
        "text": text,
        "text_source": source,
        "meta": {
            "사업명": r["사업명"],
            "발주 기관": r["발주 기관"],
            "사업 금액": r["사업 금액"],
            "공개 일자": r["공개 일자"],
            "입찰 참여 시작일": r["입찰 참여 시작일"],
            "입찰 참여 마감일": r["입찰 참여 마감일"],
            "파일형식": r["파일형식"],
            "파일명": r["파일명"],
        }
    })

# 빠른 검증
import pandas as pd
lengths = [len(d["text"]) for d in docs]
print("docs:", len(docs))
print("text_source counts:", pd.Series([d["text_source"] for d in docs]).value_counts().to_dict())
print("min/avg/max chars:", min(lengths), sum(lengths)//len(lengths), max(lengths))
print("sample:", docs[0]["doc_id"], docs[0]["text_source"], lengths[0])
print(docs[0]["text"][:500])


docs: 100
text_source counts: {'hwp_txt': 94, 'pdf_txt': 4, 'csv_text': 2}
min/avg/max chars: 1909 24869 188943
sample: 20241001798_0 hwp_txt 18149
2024. 10.

□ 사 업 명 : 한영대학교 특성화 맞춤형 교육환경 구축 - 
             트랙운영 학사정보시스템 고도화
□ 사업예산 : 130,000,000원 범위 내 (VAT 포함)
□ 사업기간 : 계약일로부터 3개월 (안정화기간 1개월 포함)
  * 기간 및 일정은 학교 사정과 용역대상자와의 협의에 따라 조정될 수 있음
□ 입찰방법 : 제한경쟁입찰(협상에 의한 계약 체결)

학사제도･제도개편과 연계하여 전공교과목 선택폭을 넓히고, 트랙제 교육과정 참여자에게 다양한 진로선택의 기회를 제공 및 취업문 확대 
트랙제 교육과정의 도입 및 운영으로 산업현장의 경쟁력 강화
산업체 수요 맞춤 교육과정 운영 및 활성화로 교육과정 내실화 
기업수요 연계 확대로 산업체 및 지역사회 현장실무형 인재 양성 

 ◦ 트랙기반 교육과정의 운영 및 관리 체계를 효과적으로 지원
 ◦ 교수자·학습자 중심의 교육환경 조성을 통한 대학 교육의 가치 구현
 ◦ 학사운영 시스템을 통해 대학 체제 개편


1.4 청킹(hy41 방법 선택)

"RFP의 의미 단위인 ‘상위 섹션’을 보존하기 위해 길이보다 섹션 구조를 우선하는 청킹을 사용했다."

- RFP 문서는 1. 사업개요 / 2. 추진배경 / 3. 사업범위처럼 번호 기반 섹션 구조가 핵심임

- 실제 질문도 **섹션 단위(“추진배경은?”, “사업범위는?”)**로 들어오는 경우가 많음

- 그래서 길이 기준이 아니라 상위 섹션(1., 2., 3.)을 최우선 경계로 청킹함

- 섹션이 너무 길 때만 보조적으로 길이 기반 분할 적용

- 150자 미만의 짧은 텍스트는 노이즈라 판단해 제거

- 결과적으로 요약·Q&A 모두에 적합하고, 임베딩 비용 대비 검색 품질이 좋은 청킹을 얻음

In [20]:
import re
from typing import List, Dict

TOP_SECTION_RE = re.compile(r"^\s*\d+\.\s+")

def split_by_top_sections(text: str) -> List[str]:
    lines = text.splitlines()
    blocks, buf = [], []
    def flush():
        nonlocal buf
        s = "\n".join(buf).strip()
        if s:
            blocks.append(s)
        buf = []
    for line in lines:
        if TOP_SECTION_RE.search(line) and buf:
            flush()
        buf.append(line)
    flush()
    return blocks

def char_chunks(text: str, chunk_size: int, chunk_overlap: int) -> List[str]:
    step = max(1, chunk_size - chunk_overlap)
    out = []
    for start in range(0, len(text), step):
        c = text[start:start+chunk_size].strip()
        if c:
            out.append(c)
        if start + chunk_size >= len(text):
            break
    return out

def hy41_chunk(text: str,
               max_chars: int = 1400,
               fallback_chunk_size: int = 1200,
               fallback_overlap: int = 200) -> List[str]:
    top_blocks = split_by_top_sections(text)
    chunks = []
    for b in top_blocks:
        if len(b) <= max_chars:
            if len(b.strip()) >= 150:
                chunks.append(b.strip())
        else:
            # top 섹션이 너무 긴 경우에만 길이 기반 분할
            chunks.extend([c for c in char_chunks(b, fallback_chunk_size, fallback_overlap) if len(c) >= 150])
    return chunks

def make_hy41(docs: List[Dict], **kwargs):
    out = []
    for d in docs:
        chunks = hy41_chunk(d["text"], **kwargs)
        for i, ch in enumerate(chunks):
            out.append({
                "doc_id": d["doc_id"],
                "chunk_id": f"{d['doc_id']}#{i}",
                "text": ch,
                "meta": d["meta"] | {"text_source": d["text_source"]},
            })
    return out

hy41 = make_hy41(docs)

lens = [len(c["text"]) for c in hy41]
print("hy41_chunks:", len(hy41))
print("min/avg/max:", min(lens), sum(lens)//len(lens), max(lens))

target_id = "20240812818_0"
tchunks = [c for c in hy41 if c["doc_id"] == target_id]
print("\nKUSF chunks:", len(tchunks))
for i in range(min(6, len(tchunks))):
    print(f"\n--- chunk #{i} ---")
    print(tchunks[i]["text"][:700])



hy41_chunks: 3324
min/avg/max: 150 753 1400

KUSF chunks: 20

--- chunk #0 ---
1. 사업 개요
 ㅇ 과 업 명: KUSF 체육특기자 경기기록 관리시스템 개발
     * 본 사업은 “소프트웨어 개발사업 적정 사업기간 산정 기준에 따른 사업” 임
 ㅇ 소요예산: 1억 5천만 원(부가가치세 포함) 
 ㅇ 사업기간: 계약체결 후 2024년 12월 31일까지
 ㅇ 사업주관: 한국대학스포츠협의회(이하 “KUSF”)
 ㅇ 사업내용: KUSF 체육특기자 경기기록 관리시스템 신규 개발

--- chunk #1 ---
2. 추진배경 및 필요성
 ㅇ (추진배경) 현행 대입 체육특기자 선발에 경기실적으로만 당락을 결정하는 문제 해결을 위한 가이드라인 및 권고 발표
     * 스포츠혁신위원회, 2019. 6. 4. 발표, 체육특기자 선발 제도 개선 2차 권고문(학교스포츠 정상화를 위한 선수육성시스템 혁신 및 일반학생의 스포츠 참여 활성화 권고)

   - 체육특기자 경기력을 평가할 수 있는 지표를 개발하여, 학생선수 개인별 경기력을 판단할 수 있는 지표 제공 추진
   - 체육특기자의 경기력 평가지표 개발 연구를 통한 농구, 배구, 야구, 축구 종목 ‘KUSF 체육특기자 경기력 평가지표’ 개발 완료(2023년)

 ㅇ (필요성) 체육특기자 경기력 평가지표 산출하고, 경기력 평가지표 산출을 위한 종목별 특성에 맞는 경기기록을 수집하기 위해 신규 시스템을 개발하여 운영하고자 함

--- chunk #2 ---
3. 사업 범위
 ㅇ 신규 개발 대상:‘KUSF 체육특기자 경기기록 관리시스템’ 
     * 관련 홈페이지·관리자페이지 모두 포함하여 개발
 ㅇ 체육특기자 경기력 평가지표 관련 대회경기기록 입력 페이지 개발
   - 대상종목: 농구, 배구, 야구, 축구 중 농구를 선별적으로 우선함 
     * 1차연도 사업으로 농구를 선별적으로 우선하고, 배구, 야구, 축구 종목으로 확대 예정
   - 대상종목 대회경기기록 및 

 사업 개요, 2. 추진배경, 3. 사업 범위가 각각 독립 청크

“중간에 짤리듯” 끊기는 느낌 거의 없음

섹션 제목 기반이라 요약/QA 둘 다에 유리

수치도 실전용으로 좋음:

3324 chunks: 약간 늘었지만 충분히 감당 가능

avg 753: 약간 짧아졌지만 700대면 괜찮은 편 (특히 섹션 단위 유지의 이득이 큼)

min 150: 노이즈 제거 유지

max 1400: 상한 유지